In [1]:
%load_ext tensorboard
import torch
import torchvision
from PIL import Image
from torchvision import transforms as T
import timm
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'numpy'